# **Part A**

* <span style="color:#daad86">**DOMAIN**: <span style="color:#659dbd"> Digital content management

* <span style="color:#daad86">**CONTEXT**: <span style="color:#659dbd">Classi ication is probably the most popular task that you would deal with in real life. Text in the form of blogs, posts, articles, etc.
are written every second. It is a challenge to predict the information about the writer without knowing about him/her. We are going to create a
classi ier that predicts multiple features of the author of a given text. We have designed it as a Multi label classi ication problem.

* <span style="color:#daad86">**DATA DESCRIPTION**: <span style="color:#659dbd">Over 600,000 posts from more than 19 thousand bloggers The Blog Authorship Corpus consists of the collected posts of
19,320 bloggers gathered from blogger.com in August 2004. The corpus incorporates a total of 681,288 posts and over 140 million words - or
approximately 35 posts and 7250 words per person. Each blog is presented as a separate ile, the name of which indicates a blogger id# and
the blogger’s self-provided gender, age, industry, and astrological sign. (All are labelled for gender and age but for many, industry and/or sign is
marked as unknown.) All bloggers included in the corpus fall into one of three age groups:
* 8240 "10s" blogs (ages 13-17)
* 8086 "20s" blogs(ages 23-27) and
* 2994 "30s" blogs (ages 33-47)
* For each age group, there is an equal number of male and female bloggers. Each blog in the corpus includes at least 200 occurrences of
common English words. All formatting has been stripped with two exceptions. Individual posts within a single blogger are separated by the
date of the following post and links within a post are denoted by the label url link.
    
* <span style="color:#daad86">**PROJECT OBJECTIVE**: <span style="color:#659dbd">To build a NLP classi ier which can use input text parameters to determine the label/s of the blog.

In [2]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
# from nltk.stem.snowball import SnowballStemmer

# import pandas_profiling as pp
import seaborn as sns
import matplotlib as plt
%matplotlib inline

In [3]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=50697923fe2cdd956777a4d0cd25a3d9a59d5878da4f65259a5168b3a822b16f
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [4]:
import langdetect

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [8]:
!pwd

/content


In [9]:
blog_df = pd.read_csv("/content/drive/My Drive/AIML/NLP/Project1/blogs/blogtext.csv")

In [10]:
blog_df.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [11]:
blog_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681284 entries, 0 to 681283
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      681284 non-null  int64 
 1   gender  681284 non-null  object
 2   age     681284 non-null  int64 
 3   topic   681284 non-null  object
 4   sign    681284 non-null  object
 5   date    681284 non-null  object
 6   text    681284 non-null  object
dtypes: int64(2), object(5)
memory usage: 36.4+ MB


There are 68,124 records which is large in size to perform analysis and computation, hence we are going to take a subset for initial setting up and rerun with entire datasource once done

In [12]:
# Creation of data sub-set
data_df=blog_df.head(15000)

# **1. Read and Analyse Dataset.**

## **A - Clearly write outcome of data analysis(Minimum 2 points)**


In [13]:
data_df.shape
print("Number of rows in the dataset: ", data_df.shape[0])
print("Number of columns in the dataset: ", data_df.shape[1])
print("Features: ", data_df.columns.tolist())
print("Number of missing values in the dataset: ", data_df.isnull().sum().values.sum())
print("Number of unique values in the dataset: ", data_df.nunique().values.sum())
print("Number of unique values in each column: ", data_df.nunique())

Number of rows in the dataset:  15000
Number of columns in the dataset:  7
Features:  ['id', 'gender', 'age', 'topic', 'sign', 'date', 'text']
Number of missing values in the dataset:  0
Number of unique values in the dataset:  16119
Number of unique values in each column:  id          353
gender        2
age          25
topic        32
sign         12
date        792
text      14903
dtype: int64


In [14]:
# Dropping non-value added variables - "id" & "date"
data_df.drop(['id','date'], axis=1, inplace=True)

In [15]:
data_df.describe()

,age
count,15000.000000
mean,26.704133
std,7.667019
min,13.000000
25%,23.000000
50%,26.000000
75%,35.000000
max,47.000000


In [16]:
# Understanding the structure of the dataset
print("Number of posts:", len(data_df))
print("Distribution of labels:")
print(data_df["age"].value_counts())

Number of posts: 15000
Distribution of labels:
age
35    2357
36    1722
17    1705
23    1540
27    1480
24    1330
25    1021
34     827
15     771
16     670
33     427
26     420
14     309
39     102
13      92
38      46
41      46
45      38
37      37
42      25
46      17
47       8
43       6
44       3
40       1
Name: count, dtype: int64


In [17]:
data_df.head()

,gender,age,topic,sign,text
0,male,15,Student,Leo,"Info has been found (+/- 100 pages,..."
1,male,15,Student,Leo,These are the team members: Drewe...
2,male,15,Student,Leo,In het kader van kernfusie op aarde...
3,male,15,Student,Leo,testing!!! testing!!!
4,male,33,InvestmentBanking,Aquarius,Thanks to Yahoo!'s Toolbar I can ...


## **B: Clean the Structured Data [3 Marks]**

**i. Missing value analysis and imputation. [1 Marks]**

In [18]:
# Check for missing values and inconsistencies
print("Missing values:")
print(data_df.isnull().sum())

Missing values:
gender    0
age       0
topic     0
sign      0
text      0
dtype: int64


In [19]:
# converting "age" from int to object
data_df['age']=data_df['age'].astype('object')

In [20]:
# explore the text
print("Average length of text per post:")
print(data_df["text"].apply(len).mean())
print("Average number of words per post:")
print(data_df["text"].apply(lambda x: len(x.split())).mean())

Average length of text per post:
1005.4659333333333
Average number of words per post:
179.22693333333333


In [21]:
# check for missing values
print(data_df.isnull().sum())

gender    0
age       0
topic     0
sign      0
text      0
dtype: int64


**ii. Eliminate Non-English textual data. [2 Marks]**

In [22]:
def is_english(text):
    try:
        lang = langdetect.detect(text)
        return lang == "en"
    except:
        return False

In [23]:
data_df = data_df[data_df["text"].apply(is_english)]

In [24]:
print("Number of posts:", len(data_df))
data_df.sample(20)

Number of posts: 14336


,gender,age,topic,sign,text
258,male,26,indUnk,Leo,Unluckily for you you have somehow land...
2709,male,35,Technology,Aries,Going to cancun before the wedding?...
6768,male,36,Fashion,Aries,So I picked up a copy of Jack Kerou...
1107,female,15,Student,Libra,Yeah. Yeah. I'd say today was a pretty...
11886,female,24,indUnk,Aries,&emailalert=no&email=adgrant@indiana.e...
5829,female,27,indUnk,Taurus,Already made it to urlLink eBay ....
5631,female,27,indUnk,Taurus,This deserves more than just a link...
3753,female,17,Student,Leo,yay!! i finally got to see brian t...
12678,male,34,Education,Pisces,............................. shocking ...
9849,female,17,Student,Taurus,You threw it all away but now your back...


In [25]:
df_txt = data_df[["text"]]

In [26]:
df_txt[:5]

,text
0,"Info has been found (+/- 100 pages,..."
2,In het kader van kernfusie op aarde...
3,testing!!! testing!!!
4,Thanks to Yahoo!'s Toolbar I can ...
5,I had an interesting conversation...


In [27]:
pd.options.mode.chained_assignment = None
df_txt['text'] = df_txt[['text']]
df_txt["text"] = df_txt["text"].astype(str)

## **Q 2. Preprocess unstructured data to make it consumable for model training. [5 Marks]**

**A. Eliminate All special Characters and Numbers [2 Marks]**

In [28]:
df_txt["text"] = df_txt["text"].apply(lambda x: re.sub(r"[^a-zA-Z ]", "", x))
df_txt.head()

,text
0,Info has been found pages and MB...
2,In het kader van kernfusie op aarde...
3,testing testing
4,Thanks to Yahoos Toolbar I can no...
5,I had an interesting conversation...


**B. Lowercase all textual data [1 Mark]**

In [29]:
df_txt["text"] = df_txt["text"].str.lower()
df_txt.head()

,text
0,info has been found pages and mb...
2,in het kader van kernfusie op aarde...
3,testing testing
4,thanks to yahoos toolbar i can no...
5,i had an interesting conversation...


**C. Remove all Stopwords [1 Marks]**

In [30]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [31]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
df_txt["text"] = df_txt["text"].apply(lambda text : remove_stopwords(text))
df_txt.head()

,text
0,info found pages mb pdf files wait untill team...
2,het kader van kernfusie op aarde maak je eigen...
3,testing testing
4,thanks yahoos toolbar capture urls popupswhich...
5,interesting conversation dad morning talking k...


**D. Remove all extra white spaces [1 Marks]**

In [32]:
df_txt['text'] = df_txt['text'].apply(lambda x: x.strip())
df_txt.head()

,text
0,info found pages mb pdf files wait untill team...
2,het kader van kernfusie op aarde maak je eigen...
3,testing testing
4,thanks yahoos toolbar capture urls popupswhich...
5,interesting conversation dad morning talking k...


Combining relevent column back

In [33]:
data_df.update(df_txt)
data_df.head(15)

,gender,age,topic,sign,text
0,male,15,Student,Leo,info found pages mb pdf files wait untill team...
2,male,15,Student,Leo,het kader van kernfusie op aarde maak je eigen...
3,male,15,Student,Leo,testing testing
4,male,33,InvestmentBanking,Aquarius,thanks yahoos toolbar capture urls popupswhich...
5,male,33,InvestmentBanking,Aquarius,interesting conversation dad morning talking k...
6,male,33,InvestmentBanking,Aquarius,somehow cocacola way summing things well early...
7,male,33,InvestmentBanking,Aquarius,anything korea country extremes everything see...
8,male,33,InvestmentBanking,Aquarius,take read news article urllink joongang ilbo n...
9,male,33,InvestmentBanking,Aquarius,surf english news sites lot looking tidbits ko...
10,male,33,InvestmentBanking,Aquarius,ah korean languageit looks difficult first fig...


Merging other columns to create a column "Labels"

In [34]:
data_df['labels']=data_df.apply(lambda col: [col['gender'],str(col['age']),col['topic'],col['sign']], axis=1)

In [35]:
data_df.head()

,gender,age,topic,sign,text,labels
0,male,15,Student,Leo,info found pages mb pdf files wait untill team...,"[male, 15, Student, Leo]"
2,male,15,Student,Leo,het kader van kernfusie op aarde maak je eigen...,"[male, 15, Student, Leo]"
3,male,15,Student,Leo,testing testing,"[male, 15, Student, Leo]"
4,male,33,InvestmentBanking,Aquarius,thanks yahoos toolbar capture urls popupswhich...,"[male, 33, InvestmentBanking, Aquarius]"
5,male,33,InvestmentBanking,Aquarius,interesting conversation dad morning talking k...,"[male, 33, InvestmentBanking, Aquarius]"


In [36]:
# Dropping columns gender,aage,topic,sign from data frame
data_df.drop(['age','gender','topic','sign'], axis=1, inplace=True)

In [37]:
data_df.head()

,text,labels
0,info found pages mb pdf files wait untill team...,"[male, 15, Student, Leo]"
2,het kader van kernfusie op aarde maak je eigen...,"[male, 15, Student, Leo]"
3,testing testing,"[male, 15, Student, Leo]"
4,thanks yahoos toolbar capture urls popupswhich...,"[male, 33, InvestmentBanking, Aquarius]"
5,interesting conversation dad morning talking k...,"[male, 33, InvestmentBanking, Aquarius]"


## **Q 3. Build a base Classification model [8 Marks]**

**A.Create dependent and independent variables [2 Marks]**
Hint: Treat ‘topic’ as a Target variable.

since for multilabel classsification we aleady created labels column, we will use that as target

In [38]:
X = data_df['text'] # independent variable
y = data_df['labels'] # dependent variable

In [39]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer=CountVectorizer(binary=True, ngram_range=(1,2))
X=vectorizer.fit_transform(X)

In [40]:
label_counts=dict()

for labels in data_df.labels.values:
    for label in labels:
        if label in label_counts:
            label_counts[label]+=1
        else:
            label_counts[label]=1

In [41]:
label_counts

{'male': 8112,
 '15': 733,
 'Student': 1631,
 'Leo': 804,
 '33': 408,
 'InvestmentBanking': 70,
 'Aquarius': 900,
 'female': 6224,
 '14': 304,
 'indUnk': 5337,
 'Aries': 4493,
 '25': 1005,
 'Capricorn': 580,
 '17': 1638,
 'Gemini': 395,
 '23': 1417,
 'Non-Profit': 142,
 'Cancer': 1092,
 'Banking': 58,
 '37': 37,
 'Sagittarius': 1522,
 '26': 384,
 '24': 1278,
 'Scorpio': 1160,
 '27': 1444,
 'Education': 602,
 '45': 36,
 'Engineering': 301,
 'Libra': 695,
 'Science': 63,
 '34': 811,
 '41': 46,
 'Communications-Media': 98,
 'BusinessServices': 135,
 'Sports-Recreation': 79,
 'Virgo': 603,
 'Taurus': 967,
 'Arts': 283,
 'Pisces': 1125,
 '44': 3,
 '16': 651,
 'Internet': 587,
 'Museums-Libraries': 64,
 'Accounting': 35,
 '39': 100,
 '35': 2169,
 'Technology': 2591,
 '36': 1679,
 'Law': 41,
 '46': 16,
 'Consulting': 56,
 'Automotive': 14,
 '42': 25,
 'Religion': 26,
 '13': 91,
 'Fashion': 1578,
 '38': 46,
 '43': 6,
 'Publishing': 65,
 '40': 1,
 'Marketing': 207,
 'LawEnforcement-Security': 4

preprocessing labels

In [42]:
from sklearn.preprocessing import MultiLabelBinarizer
binarizer=MultiLabelBinarizer(classes=sorted(label_counts.keys()))

y=binarizer.fit_transform(data_df.labels)

**B.Split data into train and test. [1 Marks]**

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=51)

In [44]:
print(X_train.shape)
print(y_train.shape)

(11468, 1002606)
(11468, 71)


In [45]:
print(X_test.shape)
print(y_test.shape)

(2868, 1002606)
(2868, 71)


Vectorize data using any one vectorizer.

In [ ]:
#Already done before splitting code


**D. Build a base model for Supervised Learning - Classification. [2 Marks]**


In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

model = LogisticRegression(solver='lbfgs')
model=OneVsRestClassifier(model)

model.fit(X_train,y_train)

OneVsRestClassifier(estimator=LogisticRegression())

**E. Clearly print Performance Metrics. [1 Marks]**
Hint: Accuracy, Precision, Recall italicised text, ROC-AUC



In [47]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

def print_evaluation_scores_micro(Ytest, Ypred):
    print('Accuracy score: ', accuracy_score(Ytest, Ypred))
    print('F1 score: ', f1_score(Ytest, Ypred, average='micro'))
    print('Average precision score: ', average_precision_score(Ytest, Ypred, average='micro'))
    print('Average recall score: ', recall_score(Ytest, Ypred, average='micro'))

def print_evaluation_scores_macro(Ytest, Ypred):
    print('Accuracy score: ', accuracy_score(Ytest, Ypred))
    print('F1 score: ', f1_score(Ytest, Ypred, average='macro'))
    print('Average precision score: ', average_precision_score(Ytest, Ypred, average='macro'))
    print('Average recall score: ', recall_score(Ytest, Ypred, average='macro'))

In [48]:
y_pred = model.predict(X_test)

In [49]:
print_evaluation_scores_micro(y_test, y_pred)

Accuracy score:  0.1903765690376569
F1 score:  0.5523410519164355
Average precision score:  0.3674125431860367
Average recall score:  0.4252092050209205


In [50]:
print_evaluation_scores_macro(y_test, y_pred)

Accuracy score:  0.1903765690376569
F1 score:  0.1791440741749523
Average precision score:  0.14256770793355802
Average recall score:  0.12921532105450811


## **5.Improve Performance of model. [14 Marks]**

**A. Experiment with other vectorisers. [4 Marks] (use tfidf)**

In [53]:
X = data_df['text'] # independent variable
y = data_df['labels'] # dependent variable

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(binary=True,ngram_range=(1,2))
X = vectorizer.fit_transform(X)


In [56]:
label_counts=dict()

for labels in data_df.labels.values:
    for label in labels:
        if label in label_counts:
            label_counts[label]+=1
        else:
            label_counts[label]=1

In [58]:
from sklearn.preprocessing import MultiLabelBinarizer
binarizer=MultiLabelBinarizer(classes=sorted(label_counts.keys()))

y=binarizer.fit_transform(data_df.labels)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)

(11468, 1002606)
(11468, 71)


**B. Build classifier Models using other algorithms than base model. [4 Marks]**(SVM)

In [60]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import LinearSVC
svcModel = LinearSVC()
svcModel = OneVsRestClassifier(svcModel, n_jobs = -1)
import time
start = time.time()

svcModel.fit(X_train, y_train)

OneVsRestClassifier(estimator=LinearSVC(), n_jobs=-1)

In [61]:
#predict on test data
y_pred = svcModel.predict(X_test)

In [62]:
print_evaluation_scores_macro(y_test, y_pred)

Accuracy score:  0.10774058577405858
F1 score:  0.10183065075571555
Average precision score:  0.10281141708412213
Average recall score:  0.07187313245860857


In [63]:
print_evaluation_scores_micro(y_test, y_pred)

Accuracy score:  0.10774058577405858
F1 score:  0.47352024922118374
Average precision score:  0.31262562472794186
Average recall score:  0.3312412831241283


**C. Tune Parameters/Hyperparameters of the model/s**

In [64]:
hyperparameters = {
    'estimator__C': [0.1, 1, 10],
    'estimator__penalty': ['l1', 'l2'],
    'estimator__loss': ['hinge', 'squared_hinge']
}

svcModel = OneVsRestClassifier(LinearSVC(), n_jobs = -1)

random_search = RandomizedSearchCV(svcModel, param_distributions=hyperparameters, n_iter=10)
random_search.fit(X_train, y_train)

RandomizedSearchCV(estimator=OneVsRestClassifier(estimator=LinearSVC(),
                                                 n_jobs=-1),
                   param_distributions={'estimator__C': [0.1, 1, 10],
                                        'estimator__loss': ['hinge',
                                                            'squared_hinge'],
                                        'estimator__penalty': ['l1', 'l2']})

In [65]:
y_pred = random_search.predict(X_test)

**D. Clearly print Performance Metrics. [2 Marks]**
    Hint: Accuracy, Precision, Recall, ROC-AUC

In [66]:
print_evaluation_scores_micro(y_test, y_pred)

Accuracy score:  0.15411436541143655
F1 score:  0.518883218252777
Average precision score:  0.3491313632352953
Average recall score:  0.37665620641562064


In [67]:
print_evaluation_scores_macro(y_test, y_pred)

Accuracy score:  0.15411436541143655
F1 score:  0.13662117671203597
Average precision score:  0.12240199567810278
Average recall score:  0.09578546915941084


Checking predictedlabel and true label

In [68]:
preds = y_pred[:15]
actuals = y_test[:15]

five_actual = binarizer.inverse_transform(actuals)
five_actual

[('36', 'Aries', 'Fashion', 'male'),
 ('35', 'Aries', 'Technology', 'male'),
 ('16', 'Capricorn', 'female', 'indUnk'),
 ('25', 'Cancer', 'Non-Profit', 'male'),
 ('24', 'Aquarius', 'Banking', 'female'),
 ('15', 'Virgo', 'indUnk', 'male'),
 ('13', 'Sagittarius', 'Student', 'female'),
 ('27', 'Taurus', 'female', 'indUnk'),
 ('23', 'Pisces', 'indUnk', 'male'),
 ('35', 'Aries', 'Technology', 'male'),
 ('24', 'Non-Profit', 'Taurus', 'male'),
 ('39', 'Education', 'Virgo', 'male'),
 ('14', 'Scorpio', 'indUnk', 'male'),
 ('35', 'Aries', 'Technology', 'male'),
 ('35', 'Aries', 'Technology', 'male')]

In [69]:
five_pred = binarizer.inverse_transform(preds)
five_pred

[('male',),
 ('male',),
 ('female',),
 ('male',),
 ('female', 'indUnk'),
 ('male',),
 ('female',),
 ('indUnk', 'male'),
 ('indUnk', 'male'),
 ('35', 'Aries', 'Technology', 'male'),
 ('female', 'indUnk'),
 ('male',),
 ('male',),
 ('male',),
 ('Aries', 'male')]

## **5.Share insights on relative performance comparison [8 Marks]**


**A. Which vectorizer performed better? Probable reason?. [2 Marks]**


> We find that the CountVectorizer outperformed the TfidfVectorizer. The performance metrics, including accuracy, F1 score, average precision score, and average recall score, were all higher for the CountVectorizer compared to the TfidfVectorizer. Hence, If these metrics are consistently higher for models using CountVectorizer, it suggests that the raw term frequencies are more effective for that particular problem.

**B. Which model outperformed? Probable reason? [2 Marks]**


> The model that utilized CountVectorizer (LogisticRegrssion) performed better the one that used TfidfVectorizer (LinearSVC).

>The observed performance difference is likely due to how each vectorization method interacts with the classifier and the specific characteristics of the dataset. The CountVectorizer with Logistic Regression could better capture the essential patterns in the text data for this particular classification task.


**C.Which  parameter/hyperparameter  significantly  helped to improve performance?Probable reason?. [2 Marks]**


> Using randomgridsearch with
hyperparameters = {
    'estimator__C': [0.1, 1, 10],
    'estimator__penalty': ['l1', 'l2'],
    'estimator__loss': ['hinge', 'squared_hinge']
}
helped in improving performance of the linearsvc. Because "randomgridsearch" will usually find a "close-enough" value in far fewer iterations.

**D.According  to  you,  which  performance  metric  should  be given most importance, why?. [2 Marks]**


> The choice of the most important performance metric is context-dependent. For imbalanced datasets and tasks where the cost of different types of errors varies, metrics like F1 score, precision, and recall are often more relevant than accuracy. For comprehensive performance evaluation across thresholds, average precision score and AUC-ROC are useful.

# **Part B**

## **Customer support chat bot**

In [70]:

import nltk
import random
import string

from nltk.chat.util import Chat, reflections
import json

In [71]:
with open('/content/drive/My Drive/AIML/NLP/Project1/GL Bot.json') as json_file:
    intents = json.load(json_file)

In [72]:
import re
pairs = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        for response in intent['responses']:
            pairs.append((pattern, response))

In [73]:
pairs

[('hi', 'Hello! how can i help you ?'),
 ('how are you', 'Hello! how can i help you ?'),
 ('is anyone there', 'Hello! how can i help you ?'),
 ('hello', 'Hello! how can i help you ?'),
 ('whats up', 'Hello! how can i help you ?'),
 ('hey', 'Hello! how can i help you ?'),
 ('yo', 'Hello! how can i help you ?'),
 ('listen', 'Hello! how can i help you ?'),
 ('please help me', 'Hello! how can i help you ?'),
 ('i am learner from', 'Hello! how can i help you ?'),
 ('i belong to', 'Hello! how can i help you ?'),
 ('aiml batch', 'Hello! how can i help you ?'),
 ('aifl batch', 'Hello! how can i help you ?'),
 ('i am from', 'Hello! how can i help you ?'),
 ('my pm is', 'Hello! how can i help you ?'),
 ('blended', 'Hello! how can i help you ?'),
 ('online', 'Hello! how can i help you ?'),
 ('i am from', 'Hello! how can i help you ?'),
 ('hey ya', 'Hello! how can i help you ?'),
 ('talking to you for first time', 'Hello! how can i help you ?'),
 ('thank you', 'I hope I was able to assist you, Goo

> 1. Start chat session with greetings and ask what the user is looking for. [5 Marks]
> 2. Accept dynamic text based questions from the user. Reply back with relevant answer from the designed corpus. [10 Marks]
> 3. End the chat session only if the user requests to end else ask what the user is looking for. Loop continues till the user asks to end it. [5 Marks]

In [75]:
!pip install fuzzywuzzy

implementation of a fuzzy string matching algorithm

In [76]:
from fuzzywuzzy import process

while True:
    request = input("User: ")
    print('User: ',request)
    match, score = process.extractOne(request, [pattern for pattern, response in pairs])
    response = next((response for pattern, response in pairs if pattern == match), None)
    if response == None:
        print("GL_BOT: I am sorry, I am not able to understand your request.")
        print("GL_BOT: Please contact the human support executive for further assistance.")
    else:
        print("GL_BOT: ", response)
    if request.lower() in ["exit", "bye", "goodbye", "see you later"]:
        print("GL_BOT: Goodbye!")
        break

User: hi
User:  hi
GL_BOT:  Hello! how can i help you ?
User: I need help
User:  I need help
GL_BOT:  Hello! how can i help you ?
User: Ive me water
User:  Ive me water
GL_BOT:  Link: Olympus wiki
User: Thank you
User:  Thank you
GL_BOT:  I hope I was able to assist you, Good Bye
User: yes
User:  yes
GL_BOT:  Link: Machine Learning wiki 
User: bye
User:  bye
GL_BOT:  I hope I was able to assist you, Good Bye
GL_BOT: Goodbye!
